In [ ]:
import pandas as pd
import json

df_inventario = None

df_empresas = pd.read_json('https://sistema-registropublicodeemissoes.fgv.br/public/organizations')

_ids = df_empresas['_id'].unique()

for _id in _ids:
    conteudo_json = urlopen('https://sistema-registropublicodeemissoes.fgv.br/public/organizations/' + 
                            str(_id)).read().decode("utf-8")
    data_json = json.loads(conteudo_json)
    
    for inv in data_json['inventories']:
        ano = inv['cycle']['inventory_year']
        nome = inv['organization']['name']
        cnpj = inv['organization']['cnpj']
        tipo = inv['organization_cycle']['organization_type']
    
        df_emissoes = pd.json_normalize(inv['emissions'])
        df_emissoes = df_emissoes[df_emissoes.computed==True].groupby(by=['scope', 'computed'], as_index=False).sum()
        df_emissoes.insert(0, column='Nome', value=nome)
        df_emissoes.insert(0, column='CNPJ', value=cnpj)
        df_emissoes.insert(0, column='Tipo', value=tipo)
        df_emissoes.insert(0, column='Ano', value=ano)

        df_emissoes.rename(columns={'scope': 'Escopo', 'value': 'tonCO2eq'}, inplace=True)
        del df_emissoes['computed']
        del df_emissoes['gas_id']
    
        if df_inventario is None:
            df_inventario = df_emissoes
        else:
            df_inventario = pd.concat([df_inventario, df_emissoes ])
    
    
    
    
        

In [ ]:
df_inventario.to_excel('inventario.xlsx', index=False)